# Aggressive Simplification with Minimal Features

**Problem**: CV-LB gap is structural. Intercept (0.0524) > Target (0.0347).

**Hypothesis**: By using minimal features, we force the model to learn more general patterns that transfer better to unseen solvents.

**Features**:
- Arrhenius kinetics: time, temp, 1/T, ln(t), interaction (5 features)
- Top 3 Spange: dielectric constant, alpha, beta (3 features)
- Total: 8 features (vs 145 in full model)

**Model**: GP(0.15) + MLP(0.55) + LGBM(0.3) ensemble with minimal features

**Baseline**: exp_035 CV 0.008194

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
print(f'Spange columns: {list(SPANGE_DF.columns)}')
print(f'Spange shape: {SPANGE_DF.shape}')

Spange columns: ['dielectric constant', 'ET(30)', 'alpha', 'beta', 'pi*', 'SA', 'SB', 'SP', 'SdP', 'N', 'n', 'f(n)', 'delta']
Spange shape: (26, 13)


In [4]:
# Minimal Featurizer - ONLY 8 features
class MinimalFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        # Only use 3 most physically meaningful Spange features
        self.spange_cols = ['dielectric constant', 'alpha', 'beta']
        self.feats_dim = 5 + 3  # kinetics (5) + spange (3) = 8 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = SPANGE_DF.loc[X["SOLVENT A NAME"]][self.spange_cols].values
            B_spange = SPANGE_DF.loc[X["SOLVENT B NAME"]][self.spange_cols].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = SPANGE_DF.loc[X["SOLVENT NAME"]][self.spange_cols].values
        
        return np.hstack([X_kinetic, X_spange])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Minimal feature dimension: {MinimalFeaturizer().feats_dim}')

Minimal feature dimension: 8


In [5]:
# Weighted Huber Loss
class WeightedHuberLoss(nn.Module):
    def __init__(self, weights=[1.0, 1.0, 2.0]):
        super().__init__()
        self.weights = torch.tensor(weights, dtype=torch.double)
        self.huber = nn.HuberLoss(reduction='none')
    
    def forward(self, pred, target):
        huber_loss = self.huber(pred, target)
        weighted_loss = huber_loss * self.weights.to(pred.device)
        return weighted_loss.mean()

# MLP Model - Smaller architecture for minimal features
class MLPModelInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[16, 8], output_dim=3, dropout=0.2):
        super(MLPModelInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, h_dim), nn.BatchNorm1d(h_dim), nn.ReLU(), nn.Dropout(dropout)])
            prev_dim = h_dim
        layers.extend([nn.Linear(prev_dim, output_dim), nn.Sigmoid()])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

print('MLP components defined')

MLP components defined


In [6]:
# MLP Ensemble with Minimal Features
class MinimalMLPEnsemble:
    def __init__(self, hidden_dims=[16, 8], n_models=5, data='single', loss_weights=[1.0, 1.0, 2.0]):
        self.hidden_dims = hidden_dims
        self.n_models = n_models
        self.data_type = data
        self.loss_weights = loss_weights
        self.featurizer = MinimalFeaturizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize_torch(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all, y_all = X_std, y_vals
            
        input_dim = X_all.shape[1]
        self.models = []
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i)
            model = MLPModelInternal(input_dim, self.hidden_dims, dropout=0.2).to(device).double()
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-3)  # Higher weight decay
            criterion = WeightedHuberLoss(self.loss_weights)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                for X_batch, y_batch in loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    optimizer.zero_grad()
                    pred = model(X_batch)
                    loss = criterion(pred, y_batch)
                    loss.backward()
                    optimizer.step()
            
            model.eval()
            self.models.append(model)

    def predict(self, X_test):
        X_std = self.featurizer.featurize_torch(X_test, flip=False).to(device)
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize_torch(X_test, flip=True).to(device)
        
        preds = []
        with torch.no_grad():
            for model in self.models:
                pred = model(X_std)
                if self.data_type == 'full':
                    pred_flip = model(X_flip)
                    pred = (pred + pred_flip) / 2
                preds.append(pred.cpu())
        
        return torch.clamp(torch.stack(preds).mean(dim=0), 0, 1)

print('MinimalMLPEnsemble defined')

MinimalMLPEnsemble defined


In [7]:
# LightGBM Wrapper with Minimal Features
class MinimalLGBM:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = MinimalFeaturizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.models = []
        params = {'objective': 'regression', 'metric': 'mse', 'boosting_type': 'gbdt',
                  'num_leaves': 15, 'learning_rate': 0.05, 'feature_fraction': 0.9,
                  'bagging_fraction': 0.8, 'bagging_freq': 5, 'verbose': -1, 'seed': 42,
                  'min_data_in_leaf': 10, 'lambda_l2': 1.0}  # More regularization
        
        for i in range(3):
            train_data = lgb.Dataset(X_all, label=y_all[:, i])
            model = lgb.train(params, train_data, num_boost_round=100)
            self.models.append(model)

    def predict(self, X_test):
        X_std = self.featurizer.featurize(X_test, flip=False)
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_test, flip=True)
        
        preds = []
        for model in self.models:
            pred = model.predict(X_std)
            if self.data_type == 'full':
                pred_flip = model.predict(X_flip)
                pred = (pred + pred_flip) / 2
            preds.append(pred)
        
        return torch.clamp(torch.tensor(np.column_stack(preds)), 0, 1)

print('MinimalLGBM defined')

MinimalLGBM defined


In [8]:
# GP Wrapper with Minimal Features
class MinimalGP:
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = MinimalFeaturizer(mixed=(data=='full'))
        self.models = []

    def train_model(self, X_train, y_train):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = y_train.values
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_std, X_flip])
            y_all = np.vstack([y_vals, y_vals])
        else:
            X_all, y_all = X_std, y_vals
        
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_all)
        
        self.models = []
        kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
        
        for i in range(3):
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True, random_state=42)
            gp.fit(X_scaled, y_all[:, i])
            self.models.append(gp)

    def predict(self, X_test):
        X_std = self.featurizer.featurize(X_test, flip=False)
        X_scaled = self.scaler.transform(X_std)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_test, flip=True)
            X_flip_scaled = self.scaler.transform(X_flip)
        
        preds = []
        for model in self.models:
            pred = model.predict(X_scaled)
            if self.data_type == 'full':
                pred_flip = model.predict(X_flip_scaled)
                pred = (pred + pred_flip) / 2
            preds.append(pred)
        
        return torch.clamp(torch.tensor(np.column_stack(preds)), 0, 1)

print('MinimalGP defined')

MinimalGP defined


In [9]:
# Minimal Feature Ensemble
class MinimalEnsemble:
    def __init__(self, data='single'):
        self.data_type = data
        self.gp = MinimalGP(data=data)
        self.mlp = MinimalMLPEnsemble(hidden_dims=[16, 8], n_models=5, data=data, loss_weights=[1.0, 1.0, 2.0])
        self.lgbm = MinimalLGBM(data=data)
        # Same weights as exp_035
        self.weights = {'gp': 0.15, 'mlp': 0.55, 'lgbm': 0.3}

    def train_model(self, X_train, y_train):
        self.gp.train_model(X_train, y_train)
        self.mlp.train_model(X_train, y_train)
        self.lgbm.train_model(X_train, y_train)

    def predict(self, X_test):
        gp_pred = self.gp.predict(X_test)
        mlp_pred = self.mlp.predict(X_test)
        lgbm_pred = self.lgbm.predict(X_test)
        
        combined = (self.weights['gp'] * gp_pred + 
                    self.weights['mlp'] * mlp_pred + 
                    self.weights['lgbm'] * lgbm_pred)
        return torch.clamp(combined, 0, 1)

print('MinimalEnsemble defined: GP(0.15) + MLP(0.55) + LGBM(0.3) with 8 features')

MinimalEnsemble defined: GP(0.15) + MLP(0.55) + LGBM(0.3) with 8 features


In [10]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MinimalEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [01:28, 88.06s/it]

2it [02:54, 87.25s/it]

3it [04:12, 82.70s/it]

4it [05:26, 79.32s/it]

...

21it [29:12, 78.58s/it]

22it [30:26, 77.25s/it]

23it [31:40, 76.47s/it]

24it [32:55, 75.74s/it]

24it [32:55, 82.29s/it]

In [11]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MinimalEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [09:18, 558.81s/it]

2it [17:47, 529.23s/it]

3it [27:14, 546.47s/it]

4it [36:51, 558.73s/it]

...

10it [1:30:33, 552.83s/it]

11it [1:40:52, 573.25s/it]

12it [1:50:44, 578.97s/it]

13it [2:00:48, 586.31s/it]

13it [2:00:48, 557.54s/it]

In [12]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [13]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE VERIFICATION ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nexp_035 baseline (145 features): CV 0.008194')
print(f'This (8 features): CV {overall_mse:.6f}')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_035!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_035')


=== CV SCORE VERIFICATION ===
Single Solvent MSE: 0.009715 (n=656)
Full Data MSE: 0.009884 (n=1227)
Overall MSE: 0.009825

exp_035 baseline (145 features): CV 0.008194
This (8 features): CV 0.009825

✗ WORSE: 19.91% worse than exp_035
